# Market Forecast based on the cycling of the one.


## *Intro*

Useful libraries:


In [54]:
from moex import MOEX
from IPython.display import Markdown as md
from pprint import pprint
from datetime import date
from get_data_file import get_filename
from basic_draw import draw_basic_plot
from urllib import request
from yahoofinancials import YahooFinancials
import json
from pathlib import Path
import pandas as pd
import csv
import os

def update_data(source:str, ticket:str, date_from:date, date_to:date, filename: str):
    match source:
        case 'moex':
            update_from_moex(filename=filename,
                ticket=ticket, date_from=date_from, date_to=date_to)
        case 'stooq':
            update_from_stooq(filename=filename,
                ticket=ticket, date_from=date_from, date_to=date_to)
        case 'msci':
            update_from_msci(filename=filename,
                ticket=ticket, date_from=date_from, date_to=date_to)
        case 'yahoo':
            update_from_yahoo(filename=filename,
                ticket=ticket, date_from=date_from, date_to=date_to)
        case _ :
            raise ValueError('Unknown data source')

def update_from_moex(filename: str, ticket: str, date_from: date, date_to: date):
    moex = MOEX()
    data = moex.history_engine_market_security(
        date_start=date_from.strftime('%YYYY%-mm-%dd'), date_end=date_to.strftime('%YYYY%-mm-%dd'), security=ticket)
    
    df = data[["TRADEDATE", "SECID", "OPEN", "CLOSE", "LOW", "HIGH", "VALUE", "CURRENCYID"]]
    df.to_csv('..\\data\\{}.csv'.format(filename))

def update_from_stooq(filename: str, ticket: str, date_from: date, date_to: date):
    date_start = date_from.strftime('%YYYY%mm%dd')
    date_end = date_to.strftime('%YYYY%mm%dd')
    url = 'https://stooq.com/q/d/l/?s={0}&d1={1}&d2={2}&i=d'.format(
        ticket, date_start, date_end)

    content = request.urlopen(url).read()

    df = pd.DataFrame(content)
    df.to_csv('..\\data\\{}.csv'.format(filename))

def update_from_msci(filename: str, ticket: str, date_from: date, date_to: date):
    date_start = date_from.strftime('%YYYY%mm%dd')  # '20200104'
    date_end = date_to.strftime('%YYYY%mm%dd')
    url = 'https://app2.msci.com/products/service/index/indexmaster/downloadLevelData?output=INDEX_LEVELS&currency_symbol=USD&index_variant=STRD&start_date={0}&end_date={1}&data_frequency=DAILY&baseValue=false&index_codes={2}'.format(
        date_start, date_end, ticket)

    content = request.urlopen(url).read()
    with open('..\\data\\msci_temp.csv', 'wb+') as fp:
        fp.write(content)

    xls = pd.ExcelFile('..\\data\\msci_temp.csv')

    sheet = xls.parse(0)

    sheet = sheet.drop(sheet.index[[0, 1, 2, 3, 4, 5]])
    sheet = sheet.drop(
        sheet.index[list(range(len(sheet.index) - 19, len(sheet.index)))])

    df = pd.DataFrame()
    df.assign(Date = sheet.column(0))
    df.assign(Close = sheet.column(1))
    df.assign(Open=pd.Series(len(sheet.column(1))))
    df.assign(Low=pd.Series(len(sheet.column(1))))
    df.assign(High=pd.Series(len(sheet.column(1))))
    df.assign(Volume=pd.Series(len(sheet.column(1))))
    df.assign(Capitalisation=pd.Series(len(sheet.column(1))))

    df.to_csv('..\\data\\{}].csv'.format(filename))

    if(os.path.isfile('..\\data\\msci_temp.csv')):
        os.remove('..\\data\\msci_temp.csv')

def update_from_yahoo(filename: str, ticket: str, date_from: date, date_to: date):
    yahoo_financials = YahooFinancials(ticket)

    historical_stock_prices = yahoo_financials.get_historical_price_data(
        start_date, end_date, step)

    df = pd.DataFrame(historical_stock_prices[ticket]['prices'])
    df.to_csv('..\\data\\{}.csv'.format(filename))


SyntaxError: invalid syntax (<ipython-input-54-00e9e99c09a9>, line 13)

Global variables:


In [ ]:
year = 2021

start_date = '1954-01-01'
end_date = date.today().isoformat()
step = 'daily'

unknown_ticket = 'UNK'

currencies = {
    'usdeur':   {'name': 'USD/EUR', 'ticket': 'usdeur', 'source': 'stooq'},
    'usdrub':   {'name': 'USD/RUB', 'ticket': 'usdrub', 'source': 'stooq'},
    'usdcny':   {'name': 'USD/CNY', 'ticket': 'usdcny', 'source': 'stooq'},
    'rubcny':   {'name': 'RUB/CNY', 'ticket': 'rubcny', 'source': 'stooq'},
    'rubeur':   {'name': 'RUB/EUR', 'ticket': 'rubeur', 'source': 'stooq'},
    #'dxy':      {'name': 'DXY', 'ticket': unknown_ticket}-
}

us_indexes = {
    'sp500':                {'name': 'S&P 500', 'ticket': '^SPX', 'source': 'stooq'},
    'dowjones':             {'name': 'Dow Jones Industrial Average', 'ticket': '^DJI', 'source': 'stooq'},
    'dowjones_transport':   {'name': 'Dow Jones Transportation Average', 'ticket': '^DJT', 'source': 'stooq'},
    'dowjones_utility':     {'name': 'Dow Jones Utility Average', 'ticket': '^DJU', 'source': 'stooq'},
    'nasdaq':               {'name': 'NASDAQ Composite', 'ticket': '^NDQ', 'source': 'stooq'},
    'russell1000':          {'name': 'Russell 1000', 'ticket': '^RUT', 'source': 'yahoo'},
    'russell2000':          {'name': 'Russell 2000', 'ticket': '^RUA', 'source': 'yahoo'},
    'russell3000':          {'name': 'Russell 3000', 'ticket': '^RUI', 'source': 'yahoo'},
    'russell2000g':         {'name': 'Russell 2000 growth', 'ticket': '^RUJ', 'source': 'yahoo'},
    'russell2000v':         {'name': 'Russell 2000 value', 'ticket': '^RUO', 'source': 'yahoo'},
    'treasury10':           {'name': 'Treasury Yield 10 Years', 'ticket': '^TNX', 'source': 'yahoo'},
    'vix':                  {'name': 'CBOE Volatility Index', 'ticket': '^VIX', 'source': 'yahoo'}
}

russia_indexies = {
    'moex':             {'name': 'MOEX Russia Index', 'ticket': 'IMOEX', 'source': 'moex'},
    'rts':              {'name': 'RTS Index USD', 'ticket': 'RTSI', 'source': 'moex'},
    'moex_bluecheaps':  {'name': 'Индекс МосБиржи голубых фишек', 'ticket': 'MOEXBC', 'source': 'moex'},
    'moex100':          {'name': 'Индекс МосБиржи широкого рынка', 'ticket': 'MOEXBMI', 'source': 'moex'},
    'moex_2and3level':  {'name': 'Индекс МосБиржи средней и малой капитализации', 'ticket': 'MCXSM', 'source': 'moex'},
    'moex_oil':         {'name': 'Индекс МосБиржи Нефти и газа', 'ticket': 'MOEXOG', 'source': 'moex'},
    'moex_electro':     {'name': 'Индекс МосБиржи Электроэнергетики', 'ticket': 'MOEXEU', 'source': 'moex'},
    'moex_telecom':     {'name': 'Индекс МосБиржи Телекоммуникаций', 'ticket': 'MOEXTL', 'source': 'moex'},
    'moex_mandm':       {'name': 'Индекс МосБиржи Металлов и добычи', 'ticket': 'MOEXMM', 'source': 'moex'},
    'moex_finance':     {'name': 'Индекс МосБиржи Финансов', 'ticket': 'MOEXFN', 'source': 'moex'},
    'moex_potreb':      {'name': 'Индекс МосБиржи Потребительского сектора', 'ticket': 'MOEXCN', 'source': 'moex'},
    'moex_chemic':      {'name': 'Индекс МосБиржи Химии и нефтехимии', 'ticket': 'MOEXCH', 'source': 'moex'},
    'moex_it':          {'name': 'Индекс МосБиржи информационных технологий', 'ticket': 'MOEXIT', 'source': 'moex'},
    'moex_builders':    {'name': 'Индекс МосБиржи строительных компаний', 'ticket': 'MOEXRE', 'source': 'moex'},
    'moex_transport':   {'name': 'Индекс МосБиржи Транспорта', 'ticket': 'MOEXTN', 'source': 'moex'},
    'moex_bond_full':   {'name': 'Индекс государственных облигаций (cовокупный доход)', 'ticket': 'RGBITR', 'source': 'moex'},
    'moex_bond':        {'name': 'Индекс государственных облигаций (ценовой)', 'ticket': 'RGBI', 'source': 'moex'},
    'moex_domclick':    {'name': 'Индекс московской недвижимости ДомКлик', 'ticket': 'MREDC', 'source': 'moex'},
    'moex_russia_vol':  {'name': 'Индекс волатильности российского рынка', 'ticket': 'RVI', 'source': 'moex'},
    'MSCI_Russia':      {'name': 'MSCI RUSSIA Standard (Large+Mid Cap)', 'ticket': '105653', 'source': 'msci'},
    'MSCI_Russia_adr':  {'name': 'MSCI RUSSIA ADR/GDR Standard(Large+Mid Cap)', 'ticket': '700044', 'source': 'msci'},
}

other_indexies = {
    #'csi':  {'name': 'China. CSI 300 Index', 'ticket': '000300.SS', 'source': 'yahoo'}, -

    'msci_china':  {'name': 'MSCI China Standard (Large+Mid Cap)', 'ticket': '302400', 'source': 'msci'},
    'msci_india':  {'name': 'MSCI India Standard (Large+Mid Cap)', 'ticket': '935600', 'source': 'msci'},
    
    'hsi':  {'name': 'China. Hang Seng Index', 'ticket': '^HSI', 'source': 'stooq'},
    'shc':  {'name': 'China. Shanghai Composite Index', 'ticket': '^SHC', 'source': 'stooq'},

    'ftse250': {'name': 'United Kingdom. FTSE 250', 'ticket': '^FTM', 'source': 'stooq'},
    'cry': {'name': 'CRB Commodity Index', 'ticket': '^CRY', 'source': 'stooq'},
    'cac40': {'name': 'France. CAC 40', 'ticket': '^CAC', 'source': 'stooq'},
    'dax': {'name': 'Germany. DAX Index', 'ticket': '^DAX', 'source': 'stooq'},

    'msci_world':  {'name': 'MSCI WORLD Standard(Large+Mid Cap)', 'ticket': '990100', 'source': 'msci'}

    #'ftse': {'name': 'Great Britain. FTSE 100', 'ticket': '^FTSE', 'source': 'stooq'},
}

md("## Point of view: {0}. Period: {1} - {2}. Step: {3}".format(
    year, start_date, end_date, step))


### Here we`ll analyze the cycling of markets.

#### Tools:

1. Seasonal pattern. Average pattern.

    + all of years.
    + Election
    + republic or democracy

2. Индекс давления ганна
3. 39 месячный цикл акций
4. Десятилетние циклы
5. Естественный цикл

#### Markets and its indexes:



### *1. Currency*


In [ ]:
md(''.join('+ {0} \({1}\). File is : {2}\n'.format(
   desc['name'],
   desc['ticket'], 
   get_filename(short_name, desc['ticket'], start_date, end_date, step))
    for short_name, desc in currencies.items() if desc['ticket'] is not unknown_ticket))


### *2. US indexies*

In [ ]:
md(''.join('+ {0} \({1}\). File is : {2}\n'.format(
   desc['name'],
   desc['ticket'],
   get_filename(short_name, desc['ticket'], start_date, end_date, step))
    for short_name, desc in us_indexes.items() if desc['ticket'] is not unknown_ticket))

### *3. Russia indexies*

In [ ]:
md(''.join('+ {0} \({1}\). File is : {2}\n'.format(
   desc['name'],
   desc['ticket'],
   get_filename(short_name, desc['ticket'], start_date, end_date, step))
    for short_name, desc in russia_indexies.items() if desc['ticket'] is not unknown_ticket))

### *4. Other indexies*

In [ ]:
md(''.join('+ {0} \({1}\). File is : {2}\n'.format(
   desc['name'],
   desc['ticket'],
   get_filename(short_name, desc['ticket'], start_date, end_date, step))
    for short_name, desc in other_indexies.items() if desc['ticket'] is not unknown_ticket))

## Currency

In [ ]:
[draw_basic_plot(filename=short_name, title=desc['ticket'])
 for short_name, desc in currencies.items() if desc['ticket'] is not unknown_ticket]

## US market

In [ ]:
[draw_basic_plot(filename=short_name, title=desc['ticket'])
 for short_name, desc in us_indexes.items() if desc['ticket'] is not unknown_ticket]


## Russia market

In [ ]:
[draw_basic_plot(filename=short_name, title=desc['ticket'])
 for short_name, desc in russia_indexies.items() if desc['ticket'] is not unknown_ticket]


## Other markets

In [ ]:
[draw_basic_plot(filename=short_name, title=desc['ticket'])
 for short_name, desc in other_indexies.items() if desc['ticket'] is not unknown_ticket]
